# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
df_all = pd.read_csv('test_data_boreholes.csv', sep=',')

## Data Checking

In [3]:
df_all.drop('Z', axis='columns', inplace=True)
df_all.rename({'new_Z':'Z'}, axis='columns', inplace=True)

## Create Boreholes from dataframes or files

In [4]:
df_all[:10]

,ID,X,Y,Z,Diameter,Length,Lithology,Colour,Thickness,Top_litho,Base_litho
0,BH001,152248.0,121422.0,102.0,0.46,37,sand,olivedrab,4,0,4
1,BH001,152248.0,121422.0,98.0,0.46,37,clay,slateblue,1,4,5
2,BH001,152248.0,121422.0,97.0,0.46,37,sandstone,blue,10,5,15
3,BH001,152248.0,121422.0,87.0,0.46,37,coal,snow,2,15,17
4,BH001,152248.0,121422.0,85.0,0.46,37,limestone,darkslategrey,20,17,37
5,BH002,152669.0,121128.0,100.0,0.33,42,sand,olivedrab,6,0,6
6,BH002,152669.0,121128.0,94.0,0.33,42,clay,slateblue,3,6,9
7,BH002,152669.0,121128.0,91.0,0.33,42,sandstone,blue,9,9,18
8,BH002,152669.0,121128.0,82.0,0.33,42,coal,snow,7,18,25
9,BH002,152669.0,121128.0,75.0,0.33,42,limestone,darkslategrey,17,25,42


In [6]:
bhs_df=[df_all]

In [7]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Lithology',
                                                lexicon=Lexicon.default(),
                                            diam_field='Diameter', 
                                            thick_field='Thickness')


Dataframe 0 processing...
|__ID:'BH001'
Error : No lithology correspondance with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=37.0)]
{0: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f41693e3d00>, 'base': <core.orm.PositionOrm object at 0x7f41693e3cd0>}, 1: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f41693e3eb0>, 'base': <core.orm.PositionOrm object at 0x7f41693e3fd0>}, 2: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f41693f3670>, 'base': <core.orm.PositionOrm object at 0x7f41693f30d0>}, 3: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f41693f31c0>, 'base': <core.orm.PositionOrm object at 0x7f41693f3130>}, 4: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f41693f3100>, 'base': <core.orm.PositionOrm object at 0x7f41693f31f0>}}

|__ID

/home/yanathan/Projects/GSDMA/utils/io.py:463: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=id_))
/home/yanathan/Projects/GSDMA/utils/io.py:463: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to Lin


Error : No lithology correspondance with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=23.0)]
{275: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f41692530d0>, 'base': <core.orm.PositionOrm object at 0x7f416926a190>}, 276: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f416926aca0>, 'base': <core.orm.PositionOrm object at 0x7f416926a1c0>}, 277: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f416926af70>, 'base': <core.orm.PositionOrm object at 0x7f416926a400>}, 278: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f416926a8b0>, 'base': <core.orm.PositionOrm object at 0x7f416926a100>}, 279: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f416926a220>, 'base': <core.orm.PositionOrm object at 0x7f416926af40>}}

|__ID:'BH057'
Error : No lithology 

|__ID:'BH114'
Error : No lithology correspondance with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=32.0)]
{565: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f416909ea30>, 'base': <core.orm.PositionOrm object at 0x7f416909ee20>}, 566: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f416909ed60>, 'base': <core.orm.PositionOrm object at 0x7f416909e550>}, 567: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f416909e370>, 'base': <core.orm.PositionOrm object at 0x7f416909e310>}, 568: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f416909e490>, 'base': <core.orm.PositionOrm object at 0x7f416909e400>}, 569: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f416909e340>, 'base': <core.orm.PositionOrm object at 0x7f416909e580>}}

|__ID:'BH115'
Error : 

|__ID:'BH163'
Error : No lithology correspondance with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=28.0)]
{810: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f4168f3c8e0>, 'base': <core.orm.PositionOrm object at 0x7f41693e3f40>}, 811: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f41693e3670>, 'base': <core.orm.PositionOrm object at 0x7f41693f3fd0>}, 812: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f416938f0d0>, 'base': <core.orm.PositionOrm object at 0x7f416938f760>}, 813: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f416938f6a0>, 'base': <core.orm.PositionOrm object at 0x7f416938f4f0>}, 814: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f416938fd30>, 'base': <core.orm.PositionOrm object at 0x7f416938f850>}}

|__ID:'BH164'
Error : 

In [8]:
boreholes

[<core.orm.BoreholeOrm>(Name=BH001, Length=37, Diameter=0.46, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH002, Length=42, Diameter=0.33, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=27, Diameter=0.42, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=27, Diameter=0.66, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=24, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH006, Length=19, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=29, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH008, Length=45, Diameter=0.62, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=24, Diameter=0.56, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=32, Diameter=0.35, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH011, Length=35, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH012, Length=19, Diameter=0.4, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH013, Length=39, Diameter=0.54, Intervals=5),
 <core.orm.Bor

In [9]:
components

{0: Component({'lithology': 'limestone'}),
 1: Component({'lithology': 'sandstone'}),
 2: Component({'lithology': 'sand'}),
 3: Component({'lithology': 'sand', 'colour': 'white'}),
 4: Component({'lithology': 'clay'})}

In [10]:
len(boreholes), len(components)

(199, 5)

## Display the striplog and 3D view

In [11]:
remove('tmp_files/tfe_orm_db.db')

In [12]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [13]:
Base.metadata.create_all(engine)

2021-06-07 16:58:37,078 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:37,080 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-07 16:58:37,080 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-07 16:58:37,082 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-07 16:58:37,083 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-07 16:58:37,084 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-07 16:58:37,085 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-07 16:58:37,086 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-07 16:58:37,087 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-07 16:58:37,088 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-07 16:58:37,088 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-07 16:58:37,089 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-07 16:58:37,090 INFO sqlalchemy.engine.Engine 

In [14]:
Session = sessionmaker(bind=engine)
session = Session()

In [15]:
p = Project(session)
p.add_components(components)

2021-06-07 16:58:37,136 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:37,140 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-07 16:58:37,141 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ()
2021-06-07 16:58:37,145 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-07 16:58:37,146 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ((0, 'Limestone'), (1, 'Sandstone'), (2, 'Sand'), (3, 'Sand, white'), (4, 'Clay'))
2021-06-07 16:58:37,148 INFO sqlalchemy.engine.Engine COMMIT
2021-06-07 16:58:37,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:37,152 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-07 16:58:37,153 INFO sqlalchemy.engi

Add boreholes into the project

In [16]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-07 16:58:37,161 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-07 16:58:37,162 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ('BH001', 37.0, 0.46)
2021-06-07 16:58:37,165 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-07 16:58:37,165 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ((0, 0.0, 0.0, 0.0, 152248.0, 121422.0), (1, 4.0, 4.0, 4.0, 152248.0, 121422.0), (2, 4.0, 4.0, 4.0, 152248.0, 121422.0), (3, 5.0, 5.0, 5.0, 152248.0, 121422.0), (4, 5.0, 5.0, 5.0, 152248.0, 121422.0), (5, 15.0, 15.0, 15.0, 152248.0, 121422.0), (6, 15.0, 15.0, 15.0, 152248.0, 121422.0), (7, 17.0, 17.0, 17.0, 152248.0, 121422.0), (8, 17.0, 17.0, 17.0, 152248.0, 121422.0), (9, 37.0, 37.0, 37.0, 152248.0, 121422.0))
2021-06-07 16:58:37,167 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, 

2021-06-07 16:58:37,271 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:37,272 INFO sqlalchemy.engine.Engine [cached since 0.09363s ago] ('BH002',)
2021-06-07 16:58:37,274 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:37,274 INFO sqlalchemy.engine.Engine [cached since 0.09066s ago] (10,)
2021-06-07 16:58:37,276 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".

2021-06-07 16:58:37,362 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:37,363 INFO sqlalchemy.engine.Engine [cached since 0.1763s ago] (23,)
2021-06-07 16:58:37,365 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:37,365 INFO sqlalchemy.engine.Engine [cached since 0.1814s ago] (24,)
2021-06-07 16:58:37,367 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

2021-06-07 16:58:37,448 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:37,448 INFO sqlalchemy.engine.Engine [cached since 0.2617s ago] (37,)
2021-06-07 16:58:37,450 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:37,451 INFO sqlalchemy.engine.Engine [cached since 0.2672s ago] (38,)
2021-06-07 16:58:37,453 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

2021-06-07 16:58:37,568 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-07 16:58:37,569 INFO sqlalchemy.engine.Engine [cached since 0.4044s ago] ((50, 0.0, 0.0, 0.0, 152669.0, 121128.0), (51, 1.0, 1.0, 1.0, 152669.0, 121128.0), (52, 1.0, 1.0, 1.0, 152669.0, 121128.0), (53, 2.0, 2.0, 2.0, 152669.0, 121128.0), (54, 2.0, 2.0, 2.0, 152669.0, 121128.0), (55, 7.0, 7.0, 7.0, 152669.0, 121128.0), (56, 7.0, 7.0, 7.0, 152669.0, 121128.0), (57, 12.0, 12.0, 12.0, 152669.0, 121128.0), (58, 12.0, 12.0, 12.0, 152669.0, 121128.0), (59, 19.0, 19.0, 19.0, 152669.0, 121128.0))
2021-06-07 16:58:37,570 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-07 16:58:37,571 INFO sqlalchemy.engine.Engine [cached since 0.4035s ago] ((25, 'BH006', 0, 'sand', 50, 51), (26, 'BH006', 1, 'clay', 52, 53), (27, 'BH006', 2, 'sandstone', 54, 55), (28, 'B

...

2021-06-07 16:58:53,180 INFO sqlalchemy.engine.Engine [cached since 15.99s ago] (1939,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
Borehole geometry created successfully !
2021-06-07 16:58:53,228 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-07 16:58:53,229 INFO sqlalchemy.engine.Engine [cached since 16.07s ago] ('BH195', 30.0, 0.65)
2021-06-07 16:58:53,230 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-07 16:58:53,231 INFO sqlalchemy.engine.Engine [cached since 16.07s ago] ((1940, 0.0, 0.0, 0.0, 152209.0, 121280.0), (1941, 1.0, 1.0, 1.0, 152209.0, 121280.0), (1942, 1.0, 1.0, 1.0, 152209.0, 121280.0), (1943, 2.0, 2.0, 2.0, 152209.0, 121280.0), (1944, 2.0, 2.0, 2.0, 152209.0, 121280.0), (1945, 6.0, 6.0, 6.0, 152209.0, 121280.0), (1946, 6.0, 6.0, 6.0, 152209.0, 121280.0), (1947, 12.0, 12.0, 12.0

2021-06-07 16:58:53,318 INFO sqlalchemy.engine.Engine COMMIT
2021-06-07 16:58:53,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:53,324 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-07 16:58:53,325 INFO sqlalchemy.engine.Engine [cached since 16.18s ago] ()
2021-06-07 16:58:53,329 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,330 INFO sqlalchemy.engine.Engine [cached since 16.15s ago] ('BH196',)
2021-06-07 16:58:53,332 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions"

2021-06-07 16:58:53,420 INFO sqlalchemy.engine.Engine [cached since 16.23s ago] (1961,)
2021-06-07 16:58:53,422 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:53,423 INFO sqlalchemy.engine.Engine [cached since 16.24s ago] (1962,)
2021-06-07 16:58:53,425 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:53,425 INFO sqlalchemy.engine.Engine [cached since 16.24s ago] (1963,)
2021-06-07 16:58:53,427 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Position

2021-06-07 16:58:53,515 INFO sqlalchemy.engine.Engine [cached since 16.33s ago] (1975,)
2021-06-07 16:58:53,516 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:53,517 INFO sqlalchemy.engine.Engine [cached since 16.33s ago] (1976,)
2021-06-07 16:58:53,519 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-07 16:58:53,520 INFO sqlalchemy.engine.Engine [cached since 16.33s ago] (1977,)
2021-06-07 16:58:53,521 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Position

2021-06-07 16:58:53,609 INFO sqlalchemy.engine.Engine [cached since 16.42s ago] (1989,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
Borehole geometry created successfully !


In [17]:
p.add_link_between_components_and_intervals(link_intv_comp)

2021-06-07 16:58:53,688 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-06-07 16:58:53,689 INFO sqlalchemy.engine.Engine [generated in 0.00316s] ((0, 2, ''), (1, 4, ''), (2, 1, ''), (3, 3, ''), (4, 0, ''), (5, 2, ''), (6, 4, ''), (7, 1, '')  ... displaying 10 of 995 total bound parameter sets ...  (993, 3, ''), (994, 0, ''))
2021-06-07 16:58:53,701 INFO sqlalchemy.engine.Engine COMMIT
2021-06-07 16:58:53,706 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:53,707 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-07 16:58:53,707 INFO sqlalchemy.engine.Engine [cached since 16.57s ago] ()


In [18]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-06-07 16:58:53,716 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  199


In [19]:
p.boreholes

2021-06-07 16:58:53,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-07 16:58:53,727 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:53,728 INFO sqlalchemy.engine.Engine [generated in 0.00135s] ('BH001',)
2021-06-07 16:58:53,730 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,730 INFO sqlalchemy.engine.Engine [cached since 16.55s ago] ('BH001',)
2021-06-07 16:58:53,732 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length"

2021-06-07 16:58:53,775 INFO sqlalchemy.engine.Engine [cached since 0.04857s ago] ('BH011',)
2021-06-07 16:58:53,777 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,777 INFO sqlalchemy.engine.Engine [cached since 16.6s ago] ('BH011',)
2021-06-07 16:58:53,779 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:53,780 INFO sqlalchemy.engine.Engine [cached since 0.05289s ago] ('BH012',)
2021-06-07 16:58:53,781 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bor

2021-06-07 16:58:53,822 INFO sqlalchemy.engine.Engine [cached since 16.64s ago] ('BH021',)
2021-06-07 16:58:53,824 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:53,824 INFO sqlalchemy.engine.Engine [cached since 0.09754s ago] ('BH022',)
2021-06-07 16:58:53,825 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,826 INFO sqlalchemy.engine.Engine [cached since 16.65s ago] ('BH022',)
2021-06-07 16:58:53,826 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".leng

2021-06-07 16:58:53,852 INFO sqlalchemy.engine.Engine [cached since 0.1252s ago] ('BH032',)
2021-06-07 16:58:53,853 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,853 INFO sqlalchemy.engine.Engine [cached since 16.68s ago] ('BH032',)
2021-06-07 16:58:53,855 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:53,856 INFO sqlalchemy.engine.Engine [cached since 0.1291s ago] ('BH033',)
2021-06-07 16:58:53,858 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bore

2021-06-07 16:58:53,905 INFO sqlalchemy.engine.Engine [cached since 16.73s ago] ('BH042',)
2021-06-07 16:58:53,907 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:53,907 INFO sqlalchemy.engine.Engine [cached since 0.1807s ago] ('BH043',)
2021-06-07 16:58:53,909 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:53,909 INFO sqlalchemy.engine.Engine [cached since 16.73s ago] ('BH043',)
2021-06-07 16:58:53,911 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".lengt

...

2021-06-07 16:58:54,414 INFO sqlalchemy.engine.Engine [cached since 0.6876s ago] ('BH158',)
2021-06-07 16:58:54,416 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:54,417 INFO sqlalchemy.engine.Engine [cached since 17.24s ago] ('BH158',)
2021-06-07 16:58:54,418 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:54,419 INFO sqlalchemy.engine.Engine [cached since 0.6925s ago] ('BH159',)
2021-06-07 16:58:54,421 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bore

2021-06-07 16:58:54,463 INFO sqlalchemy.engine.Engine [cached since 17.29s ago] ('BH168',)
2021-06-07 16:58:54,465 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:54,466 INFO sqlalchemy.engine.Engine [cached since 0.7392s ago] ('BH169',)
2021-06-07 16:58:54,467 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:54,468 INFO sqlalchemy.engine.Engine [cached since 17.29s ago] ('BH169',)
2021-06-07 16:58:54,470 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".lengt

2021-06-07 16:58:54,516 INFO sqlalchemy.engine.Engine [cached since 0.789s ago] ('BH179',)
2021-06-07 16:58:54,517 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:54,517 INFO sqlalchemy.engine.Engine [cached since 17.34s ago] ('BH179',)
2021-06-07 16:58:54,518 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:54,519 INFO sqlalchemy.engine.Engine [cached since 0.792s ago] ('BH180',)
2021-06-07 16:58:54,520 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".boreho

2021-06-07 16:58:54,555 INFO sqlalchemy.engine.Engine [cached since 17.38s ago] ('BH189',)
2021-06-07 16:58:54,557 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-07 16:58:54,558 INFO sqlalchemy.engine.Engine [cached since 0.831s ago] ('BH190',)
2021-06-07 16:58:54,559 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-07 16:58:54,559 INFO sqlalchemy.engine.Engine [cached since 17.38s ago] ('BH190',)
2021-06-07 16:58:54,561 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length

[<core.orm.BoreholeOrm>(Name=BH001, Length=37.0, Diameter=0.46, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH002, Length=42.0, Diameter=0.33, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=27.0, Diameter=0.42, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=27.0, Diameter=0.66, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=24.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH006, Length=19.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=29.0, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH008, Length=45.0, Diameter=0.62, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=24.0, Diameter=0.56, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=32.0, Diameter=0.35, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH011, Length=35.0, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH012, Length=19.0, Diameter=0.4, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH013, Length=39.0, Diameter=0.54, In

In [20]:
p.refresh(verbose=True)

2021-06-07 16:58:54,599 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-07 16:58:54,600 INFO sqlalchemy.engine.Engine [cached since 17.46s ago] ()


In [21]:
session.close()

2021-06-07 16:58:54,608 INFO sqlalchemy.engine.Engine ROLLBACK


## Display one object

In [22]:
i = 'BH001'
df_all.query(f"ID=='{i}'")#['Thickness'].cumsum().max()

,ID,X,Y,Z,Diameter,Length,Lithology,Colour,Thickness,Top_litho,Base_litho
0,BH001,152248.0,121422.0,102.0,0.46,37,sand,olivedrab,4,0,4
1,BH001,152248.0,121422.0,98.0,0.46,37,clay,slateblue,1,4,5
2,BH001,152248.0,121422.0,97.0,0.46,37,sandstone,blue,10,5,15
3,BH001,152248.0,121422.0,87.0,0.46,37,coal,snow,2,15,17
4,BH001,152248.0,121422.0,85.0,0.46,37,limestone,darkslategrey,20,17,37


In [23]:
bh = p.boreholes_3d[0]
print(bh.name, '-', bh.intervals) # problem with top and base position --> kind of overlaying

BH001 - [Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 4.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'sand', 'data': {}, 'components': [Component({})]}), Interval({'top': Position({'middle': 4.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 5.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'clay', 'data': {}, 'components': [Component({})]}), Interval({'top': Position({'middle': 5.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 15.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'sandstone', 'data': {}, 'components': [Component({})]}), Interval({'top': Position({'middle': 15.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 17.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'white sand', 'data': {}, 'components': [Component({})]}), Interval(

In [24]:
p.boreholes_3d[0].plot3d(x3d=True)

In [25]:
pause

NameError: name 'pause' is not defined

### Display project boreholes

### Boreholes in 3D using X3D for interaction within the notebook

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)